# Content-based search

The Internet has brought forward a marvelous source of information. But - simply knowing that we *have* information is just not enough to *use* this information. For example, we *know* that, somewhere on the Internet, there is a book on Natural Language Processing. But, how can we find this book?

In this notebook, we are going to work with the following use case (which was also approached in [Amami et al., "An LDA-Based Approach to Scientific Paper Recommendation",Natural Language Processing and Information Systems, 2016 ](http://link.springer.com/10.1007/978-3-319-41754-7_17), based on ideas by [Griffiths and Steyvers, "Finding Scientific Topics", Proc. Natl. Acad. Sci. U.S.A., 2004](https://doi.org/10.1073/pnas.0307752101).

Suppose a scientist is writing an article. Articles usually start with a session called "abstract", which summarizes the contents of the whole paper. We want our system to get the abstract we are working with, and then find possible articles we could work with.

We will start by simulating our data with a subset of an ArXiv dataset available at Kaggle:

In [2]:
import pandas as pd 
import os
import kagglehub
from tqdm import tqdm
from pathlib import Path
    
path = kagglehub.dataset_download("tiagoft/arvix-data-filtered-for-cs-only-data")
path = Path(path)
df = pd.read_csv(path / 'arxiv-metadata-oai-snaptshot-cs-only.csv')

In [3]:
sample_title = "Enhancing Autonomous Agents with Multimodal Generative AI for Improved Human-AI Collaboration"
sample_abstract = """The integration of multimodal generative AI into autonomous agents presents a significant advancement in human-AI collaboration. 
This study explores the development of autonomous agents capable of processing and generating various data types,
including text-to-image and image-to-audio conversions. By leveraging multimodal generative AI, these agents can interpret and generate 
content across different modalities, enhancing their ability to interact with humans in more natural and intuitive ways.
We propose a novel framework that combines generative AI with transfer learning techniques to enable autonomous agents to adapt 
knowledge acquired from one context to another with minimal additional data. Our experiments demonstrate that this approach significantly
improves the agents' performance in tasks requiring human-AI collaboration, such as virtual reality environments and smart city applications.
The results highlight the potential of multimodal generative AI to revolutionize human-AI interaction, paving the way for more immersive 
and adaptive collaborative experiences.
"""
sample_keywords = ["autonomous agents", "multimodal generative AI", "human-AI collaboration", "transfer learning", "virtual reality", "smart city applications"]

## Exercise 1: search by keyword

Searching by keywords is somewhat simple because we can simply use an inverted index. In fact, online search engines usually implement inverted index.

Use your inverted index to try to find other, relevant articles within our dataset using the keywords provided by the abstract's author.

In [4]:
import re
from collections import defaultdict
from nltk.stem import WordNetLemmatizer

iidx = defaultdict(list)

def add_no_indice(iidx, new_text, idx, lemmatize=False):
    text = new_text.lower()
    words = re.findall(r"\b\w\w+\b", text)

    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        words = [ lemmatizer.lemmatize(word) for word in words ]

    for word in words:
        iidx[word].append(idx)
    
    return iidx
        
def encontrar_no_indice(iidx, word, lemmatize=False):
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        word = lemmatizer.lemmatize(word)

    return iidx[word]

NameError: name 'idx' is not defined

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(df['abstract'])

doc_freq_matrix = X.mean(axis=0)

In [ ]:
inverted_idx = {}
words = sorted(vectorizer.vocabulary_)
for i in range(doc_freq_matrix.shape[0]):
    for j in range(doc_freq_matrix.shape[1]):
        word = 
        inverted_idx[vectorizer.vocabulary_[i]] = inverted_idx.get(vectorizer.vocabulary_[i], []) + [j]

In [ ]:
inverted_idx

## Exercise 2: finding better keywords

Keywords are words that differentiate a particular document from the other documents in the collection.

This means that the TFIDF measure could be useful to find keywords within a document.

For such, fit a TFIDF vectorizer in the whole collection of abstracts and then experiment to find out:

1. if the words with largest TFIDF in our abstract are the same as the proposed keywords
1. if the words are meaningful towards our abstract
1. if searching by the TFIDF-generated words could lead to better recommendations

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
  min_df=2, # filtro de regularização para palavras que aparecem em ao menos de x documentos
  max_df=0.8, # filtro de regularização para palavras que aparecem em mais de x% dos documentos
  max_features=10000, # limite de palavras no vocabulário
  stop_words='english'
)

X = vectorizer.fit_transform(df['abstract']).toarray()

MemoryError: Unable to allocate 4.69 GiB for an array with shape (62905, 10000) and data type float64

## Exercise 3: modelling abstracts with topics

Remember that, in our topic model with LDA, we decompose the word count matrix as:

$$
X \approx BA,
$$

where $B$ contains a representation of each document in terms of its topics.

However, we have not discussed how to find an optimal number of topics.

The idea used by [Amami et al.](http://link.springer.com/10.1007/978-3-319-41754-7_17) is to choose the number of topics that minimizes a metric called *perplexity*.

Perplexity is a measure of the certainty of sampling a word using our model (see [Griffiths and Steyvers](https://doi.org/10.1073/pnas.0307752101)). Lower values are better. With too few topics, the model is in fact making very broad assumptions regarding data; with too many topics, there is a greater chance of finding data is too sparse for a relevant estimation.

Modify the code below to find an optimal number of topics for our data. Then, decompose all documents in the collection (also, do it to our abstract!) using the topic model.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from tqdm import tqdm

print('Fitting vectorizer')
vectorizer = CountVectorizer(stop_words='english', min_df=10, max_df=0.8, max_features=1000).fit(df['abstract'])
abstract_vectorized = vectorizer.transform(df['abstract'].sample(10000))

print('Fitting LDA')
for n_components in tqdm([2, 10, 20, 50, 100]):
    lda = LatentDirichletAllocation(n_components=n_components, random_state=42, n_jobs=-1)
    lda.fit(abstract_vectorized)
    print(f"Number of components: {n_components}. Perplexity: {lda.perplexity(abstract_vectorized)}")


## Exercise 4: KL and JS divergences

The decomposition resulting from LDA is a probability distribution. The distance between two probability distributions can be calculated using the Kullback-Leibner divergence, which is calculated by:

$$
D_{KL}(P \parallel Q) = \sum_{i} P(i) \log \left( \frac{P(i)}{Q(i)} \right)
$$

However, the KL divergence is not symetric, which was bothersome to Amani and their colleagues. For this reason, they used the Jensen-Shannon (JS) divergence, given by:

$$
D_{JS}(P,Q) = \frac{D_{KL}(P \parallel Q) + D_{KL}(Q \parallel P)}{2}
$$

See the code below demonstrating how this works in practice:


In [ ]:
from scipy.spatial.distance import jensenshannon

lda = LatentDirichletAllocation(n_components=5, random_state=42, n_jobs=-1)
lda.fit(abstract_vectorized)

topics1 = lda.transform(abstract_vectorized[0,:])
topics2 = lda.transform(abstract_vectorized[1,:])
topics3 = lda.transform(abstract_vectorized[500,:])

print(topics1)
print(topics2)
print(topics3)

print(jensenshannon(topics1.ravel(), topics2.ravel()))
print(jensenshannon(topics1.ravel(), topics3.ravel()))
print(jensenshannon(topics2.ravel(), topics3.ravel()))

Using the LDA models you fitted in Exercise 4. Find the topic models for our abstract, and for each of the elements in the dataset. Then, make a function that retrieves the $K$ elements (where $K$ is an integer you can choose!) from the dataset that are closer to our abstract!

## Exercise 5

Compare the recommendations provided by keyword searching, by TDIDF keyword searching, and by topic modelling. 

1. Which recommendation seems more useful?
1. Could you combine the techniques above (at least 2 of them) to get a possibly better recommendation?
1. Can you use an LLM to help with this task? How? Implement an LLM-based solution and compare it with the previous ones.
